In [1]:
# Import findspark and initialise. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/2/athlete_events.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("athlete_events.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summ

In [18]:
print(f"Number of rows: {format((df.count()), ',')}.")

Number of rows: 271,116.


In [19]:
#Create a temporary view.
df.createOrReplaceTempView('games')

In [20]:
# Get the number of distinct sports played in 2012
spark.sql("""
          SELECT COUNT(DISTINCT sport) AS 2016_sports
          FROM games
          WHERE Year = 2012
          """).show()

+-----------+
|2016_sports|
+-----------+
|         32|
+-----------+



In [21]:
# Sports in 2016
spark.sql("""
            SELECT DISTINCT sport
            FROM games
            WHERE Year = 2016
          """).show(3)

+----------+
|     sport|
+----------+
|Gymnastics|
|    Tennis|
|    Boxing|
+----------+
only showing top 3 rows



In [22]:
# Sports in 2012
spark.sql("""
            SELECT DISTINCT sport
            FROM games
            WHERE Year = 2012
          """).show(3)

+----------+
|     sport|
+----------+
|Gymnastics|
|    Tennis|
|    Boxing|
+----------+
only showing top 3 rows



In [23]:
# Write a subquery to get the distinct sports played in 2016 but not in 2012.
spark.sql("""
            SELECT DISTINCT sport
            FROM games
            WHERE Year = 2016 AND sport NOT IN
            (SELECT DISTINCT sport
            FROM games
            WHERE Year = 2012)
          """).show()

+------------+
|       sport|
+------------+
|        Golf|
|Rugby Sevens|
+------------+



In [24]:
# Get the year of the earliest Olympics in our data
spark.sql("""
            SELECT year
            FROM games
            ORDER BY year ASC
            LIMIT 1
        """).show()

+----+
|year|
+----+
|1896|
+----+



In [26]:
# Get the year of the earliest Olympics in our data
# Alternative using min
spark.sql("""
            SELECT MIN(year)
            FROM games
        """).show()


+---------+
|min(year)|
+---------+
|     1896|
+---------+



In [25]:
# Get the sports that were played in the earliest Olympic games in our data.
# Using subqueries
spark.sql("""
            SELECT DISTINCT sport
            FROM games
            WHERE Year = (SELECT MIN(Year) FROM games)
            ORDER BY sport ASC
          """).show()

+-------------+
|        sport|
+-------------+
|    Athletics|
|      Cycling|
|      Fencing|
|   Gymnastics|
|     Shooting|
|     Swimming|
|       Tennis|
|Weightlifting|
|    Wrestling|
+-------------+



In [28]:
# Who are the top 5 Gold medal winners in the Olympics
top5_gold = spark.sql("""
          SELECT Name, Medal, COUNT(Medal) AS MedalCount
          FROM games
          WHERE Medal = 'Gold'
          GROUP BY Name, Medal
          SORT BY COUNT(Medal) DESC
          LIMIT 5
          """)

top5_gold.show()

+--------------------+-----+----------+
|                Name|Medal|MedalCount|
+--------------------+-----+----------+
|Michael Fred Phel...| Gold|        23|
|Raymond Clarence ...| Gold|        10|
|Frederick Carlton...| Gold|         9|
|Larysa Semenivna ...| Gold|         9|
|Paavo Johannes Nurmi| Gold|         9|
+--------------------+-----+----------+



In [29]:
# Who are the top 5 Silver medal winners
spark.sql("""
          SELECT Name, Medal, COUNT(Medal) AS MedalCount
          FROM games
          WHERE Medal = 'Silver'
          GROUP BY Name, Medal
          SORT BY COUNT(Medal) DESC
          LIMIT 5
          """).show()

+--------------------+------+----------+
|                Name| Medal|MedalCount|
+--------------------+------+----------+
|Shirley Frances B...|Silver|         6|
|Aleksandr Nikolay...|Silver|         6|
|Aleksandr Vladimi...|Silver|         6|
|Mikhail Yakovlevi...|Silver|         6|
|           Yang Yang|Silver|         6|
+--------------------+------+----------+



In [30]:
# Who are the top 5 Bronze medal winners
spark.sql("""
          SELECT Name, Medal, COUNT(Medal) AS MedalCount
          FROM games
          WHERE Medal = 'Bronze'
          GROUP BY Name, Medal
          SORT BY COUNT(Medal) DESC
          LIMIT 5
          """).show()


+--------------------+------+----------+
|                Name| Medal|MedalCount|
+--------------------+------+----------+
|Harri Tapani Kirv...|Bronze|         6|
|Merlene Joyce Ott...|Bronze|         6|
|Aleksey Yuryevich...|Bronze|         6|
|Heikki Ilmari Sav...|Bronze|         6|
|Franziska van Alm...|Bronze|         6|
+--------------------+------+----------+



In [31]:
import pandas as pd
top5_gold.toPandas().to_csv('top5Gold.csv')